In [42]:
import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom
from numba import jit
import matplotlib
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [26]:
df = pd.read_csv('export.csv')

In [27]:
df_next_event = df.copy()

#A function for determining the true next event for each event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]

        idx+=1
    return res

df_next_event['next_event'] = calculator_nb(df_next_event['case'].values, df_next_event['event'].values)

C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\3644305937.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_7684\3644305937.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\3644305937.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_7684\3644305937.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_nb" w

In [28]:
df = df_next_event

In [29]:
goodshit = ['case','event','number_parcels','payment_actual0','area','cross_compliance','penalty_amount0','next_event']
df_reduced = df.copy()

for col in df.columns:
    if (goodshit.count(col) == 0):
        df_reduced = df_reduced.drop(col, 1)

C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\384

C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_7684\384

In [30]:
def normalize(col_name):
    col_as_array = df_reduced[col_name].to_numpy()
    col_as_array = np.where(col_as_array == 0, 0.5, col_as_array)
    col_as_array_norm = np.log10(col_as_array)
    mean = col_as_array_norm.mean()
    stdev = col_as_array_norm.std()
    epsilon = 0.01
    return (col_as_array_norm - mean) / (stdev + epsilon)

In [11]:
def col_to_millis(col_name):
    col_as_array = df_reduced[col_name].to_numpy()
    col_as_array[:] = [elem[:19] for elem in col_as_array]
    
    result = []
    for date in col_as_array:
        utctime = datetime.strptime(date, "%Y/%m/%d %H:%M:%S")
        result.append((utctime - datetime(1970, 1, 1)) // timedelta(milliseconds=1))
    
    return result

In [31]:
df_reduced = df_reduced.fillna(value="None")

In [32]:
df_reduced, validate, test = np.split(df_reduced.sample(frac=1), [int(.6*len(df_reduced)), int(.8*len(df_reduced))])

In [33]:
event = df_reduced['event'].to_numpy()
next_event = df_reduced['next_event'].to_numpy()

In [48]:
ordinal_encoder = OrdinalEncoder()
label_encoder = LabelEncoder()
event = event.reshape(-1, 1)
event_encoded = ordinal_encoder.fit_transform(event)
next_event_encoded = label_encoder.fit_transform(next_event)
next_event_encoded = next_event_encoded.reshape(-1, 1)

C:\Users\mikok\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[39],
       [20],
       [ 0],
       ...,
       [ 9],
       [ 0],
       [18]])

In [49]:
event_encoded

array([[24.],
       [ 8.],
       [20.],
       ...,
       [17.],
       [20.],
       [19.]])

In [50]:
number_parcels = normalize('number_parcels')
payment_actual0 = normalize('payment_actual0')
area = normalize('area')
cross_compliance = normalize('cross_compliance')
penalty_amount0 = normalize('penalty_amount0')

In [51]:
training = []
for i in range(len(event_encoded)):
    current = event_encoded[i]
    current = np.append(current,number_parcels[i])
    current = np.append(current,payment_actual0[i])
    current = np.append(current,area[i])
    current = np.append(current,cross_compliance[i])
    current = np.append(current,penalty_amount0[i])
    training.append(current)

In [54]:
labels = []
for i in range(len(next_event_encoded)):
    current = next_event_encoded[i]
    current = np.append(current,[0,0,0,0,0])
    labels.append(current)

In [55]:
labels

[array([39,  0,  0,  0,  0,  0]),
 array([20,  0,  0,  0,  0,  0]),
 array([0, 0, 0, 0, 0, 0]),
 array([39,  0,  0,  0,  0,  0]),
 array([7, 0, 0, 0, 0, 0]),
 array([28,  0,  0,  0,  0,  0]),
 array([39,  0,  0,  0,  0,  0]),
 array([20,  0,  0,  0,  0,  0]),
 array([5, 0, 0, 0, 0, 0]),
 array([9, 0, 0, 0, 0, 0]),
 array([20,  0,  0,  0,  0,  0]),
 array([9, 0, 0, 0, 0, 0]),
 array([20,  0,  0,  0,  0,  0]),
 array([5, 0, 0, 0, 0, 0]),
 array([5, 0, 0, 0, 0, 0]),
 array([20,  0,  0,  0,  0,  0]),
 array([27,  0,  0,  0,  0,  0]),
 array([24,  0,  0,  0,  0,  0]),
 array([39,  0,  0,  0,  0,  0]),
 array([5, 0, 0, 0, 0, 0]),
 array([30,  0,  0,  0,  0,  0]),
 array([39,  0,  0,  0,  0,  0]),
 array([5, 0, 0, 0, 0, 0]),
 array([24,  0,  0,  0,  0,  0]),
 array([21,  0,  0,  0,  0,  0]),
 array([24,  0,  0,  0,  0,  0]),
 array([5, 0, 0, 0, 0, 0]),
 array([39,  0,  0,  0,  0,  0]),
 array([9, 0, 0, 0, 0, 0]),
 array([5, 0, 0, 0, 0, 0]),
 array([5, 0, 0, 0, 0, 0]),
 array([20,  0,  0,  0, 

In [56]:
training

[array([24.        ,  0.67195516,  0.67781942,  0.75199694, -0.16502902,
         0.2940764 ]),
 array([ 8.        ,  0.46107935,  0.3383404 ,  0.37645017, -0.16502902,
         2.17574642]),
 array([20.        , -0.46075689, -0.34572904, -0.38733137, -0.16502902,
        -0.78958545]),
 array([35.        , -1.51193966, -1.96328115, -2.07708018, -0.16502902,
        -0.78958545]),
 array([17.        ,  1.04269327,  0.83975852,  0.84996356, -0.16502902,
        -0.78958545]),
 array([38.        ,  0.37710017,  0.2420941 ,  0.22968351, -0.16502902,
         0.96544356]),
 array([27.        ,  0.23290208,  0.75571532,  0.82724122, -0.16502902,
         1.24013713]),
 array([ 8.        ,  0.57312441, -0.02360691, -0.0642753 , -0.16502902,
         0.13510659]),
 array([38.        ,  1.62430718,  2.51493159,  1.84113217, -0.16502902,
         1.06669566]),
 array([ 4.        ,  0.84033623,  0.62981571,  0.69803209, -0.16502902,
        -0.78958545]),
 array([19.        , -1.17171733, -1.007

In [57]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(43, activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [58]:
model.fit(training,labels,epochs=1,verbose=2)

KeyboardInterrupt: 